In [45]:
import alpaca_trade_api as tradeapi
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras import callbacks
from sklearn.model_selection import train_test_split
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from IPython.display import clear_output
import datetime
import statistics
import time
import logging
import os
import json
import yfinance as yf
from keras.models import model_from_json
import requests
from keras.models import load_model
from matplotlib import pyplot as plt

In [46]:
# Set your API key and secret
api_key = 'ur keyapi_secret = 'ur secretbase_url = 'https://paper-api.alpaca.markets'  # Use paper trading base URL for testing

# Initialize Alpaca API
api = tradeapi.REST(api_key, api_secret, base_url, api_version='v2')

# Get historical market data
symbol = 'AAPL'  # Replace with your desired stock symbol
timeframe = '1D'  # Daily timeframe
historical_data = api.get_bars(symbol, timeframe).df
historical_data.head()

,close,high,low,trade_count,open,volume,vwap
timestamp,,,,,,,
2023-12-08 05:00:00+00:00,195.89,195.99,193.67,515247,194.2,38895669,194.9612


In [47]:
def get_data(symbol,data_len,seq_len):
    end = datetime.datetime.today().strftime('%Y-%m-%d')
    start = datetime.datetime.strptime(end, '%Y-%m-%d') - datetime.timedelta(days=(data_len/0.463))
    stock_data = yf.download(symbol,start,end)
    close = stock_data['Close'].values
    open_ = stock_data['Open'].values
    high = stock_data['High'].values
    low = stock_data['Low'].values
    symbol_data, minmax = normal_data(stock_data)
    cols = symbol_data.columns.tolist()

    symbol_seq = list()
    for i in range(len(cols)):
        if cols[i] < 4:
            symbol_seq.append(symbol_data[cols[i]].values)
            symbol_seq[i] = symbol_seq[i].reshape((len(symbol_seq[i]), 1))
    data = hstack(symbol_seq)
    n_steps = seq_len
    X, y = split_sequences(data, n_steps)
    n_features = X.shape[2]
    n_seq = len(X)
    n_steps = seq_len
    print(X.shape)
    X = X.reshape((n_seq, 1, n_steps, n_features))
    true_y = []
    for i in range(len(y)):
        true_y.append([y[i][0], y[i][1]])
    return X, array(true_y), n_features, minmax, n_steps, close, open_, high, low

In [48]:
def split_sequences(seq, n_steps):
        X, y = list(), list()
        for i in range(len(seq)):
            end_idx = i + n_steps
            if end_idx > len(seq) - 1:
                break
            seq_x, seq_y = seq[i:end_idx, :], seq[end_idx, :]
            X.append(seq_x)
            y.append(seq_y)
        return array(X), array(y)

In [49]:
def normal_data(stock_data):
        cols = stock_data.columns.tolist()
        col_name = [0] * len(cols)
        for i in range(len(cols)):
            col_name[i] = i
        stock_data.columns = col_name
        value_ranges = list()
        for column in stock_data:
            symbol_data = symbol_data.astype({column: 'float32'})
        for i in range(len(cols)):
            col_values = symbol_data[col_name[i]]
            value_min = min(col_values)
            value_max = max(col_values)
            value_ranges.append([value_min, value_max])
        for column in symbol_data:
            values = symbol_data[column].values
            for i in range(len(values)):
                values[i] = (values[i] - value_ranges[column][0]) / (value_ranges[column][1] - value_ranges[column][0])
            symbol_data[column] = values
        symbol_data[column] = values
        return symbol_data, value_ranges

In [50]:
def split_data(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, seed = 7777)
    return X_train, X_test, y_train, y_test

In [51]:
def run_lstm(n_steps,n_features,optimizer = 'Adam'):
    lstm_mod = Sequential()
    lstm_mod.add(TimeDistributed(Conv1D(filters = 64, kernel_size = 1, activation = 'relu'), input_shape=(None, n_steps, n_features)))
    lstm_mod.add(TimeDistributed(MaxPooling1D(pool_size = 2)))
    lstm_mod.add(TimeDistributed(Flatten()))
    lstm_mod.add(LSTM(50, activation = 'relu'))
    lstm_mod.add(Dense(2))
    lstm_mod.compile(optimizer = optimizer, loss = 'mse')
    return lstm_mod

In [52]:
def train_model(X_train, y_train, model, epochs):
    dirx = 'C:\\Users\\85762\\OneDrive\\Desktop\\FINM 325'
    os.chdir(dirx)
    h5 = 'Stocks'+'_best_model'+'.h5'
    checkpoint = callbacks.ModelCheckpoint(h5, monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='auto', period=1)
    earlystop = callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=epochs * 1/4, verbose=0, mode='auto', baseline=None,restore_best_weights=True)
    callback = [earlystop, checkpoint]
    json = 'Stocks'+'_best_model'+'.json'
    model_json = model.to_json()
    with open(json, "w") as json_file:
        json_file.write(model_json)
    history = model.fit(X_train, y_train, epochs = epochs, batch_size=len(X_train)//4, verbose = 2,validation_split = 0.3, callbacks = callback)
    return history

In [53]:
def load_model(dataset, model, loss, optimizer):
    dirx = 'C:\\Users\\85762\\OneDrive\\Desktop\\FINM 325'
    os.chdir(dirx)
    json_file = open(dataset+'_best_model'+'.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    model.compile(optimizer = optimizer, loss = loss, metrics = None)
    model.load_weights(dataset+'_best_model'+'.h5')
    return model

In [54]:
def model_predict(model, value_ranges, n_features, n_steps, data, test_loss):
    new_data = data[-1].reshape((len(data[-1]), 1, n_steps, n_features))
    prediction = model.predict(new_data)[0]
    predict_loss = list()
    for i in range(len(prediction)):
        prediction[i] = prediction[i] * (value_ranges[i][1] - value_ranges[i][0]) + value_ranges[i][0]
        predict_loss.append(((100-test_loss) / 100) * (value_ranges[i][1] - value_ranges[i][0]))
    return prediction, predict_loss

In [55]:
def create_order(prediction, company, test_loss, predict_loss):
    open_price, close_price = prediction[0], prediction[1]
    if open_price > close_price:
        side = 'sell'
    elif open_price < close_price:
        side = 'buy'
    if side == 'buy':
        order = {
            'symbol': company,
            'quantity': round(20 * (test_loss / 100)),
            'type': 'stop_limit',
            'time_in_force': 'day',
            'side': 'buy',
            'take_profit': close_price + predict_loss,
            'stop_loss': close_price - predict_loss
                }
    elif side == 'sell':
        order = {
            'symbol': company,
            'quantity': round(20 * (test_loss / 100)),
            'type': 'stop_limit',
            'time_in_force': 'day',
            'side': 'sell',
            'take_profit': close_price - predict_loss,
            'stop_loss': close_price + predict_loss
                }
    r = requests.post(ORDERS_URL, json = order, headers = HEADERS)
    print(r.content)

In [ ]:
symbol = "AAPL"
interval = 60
data_len = 365
seq_len = 60
epochs = 10

BASE_URL = 'https://paper-api.alpaca.markets'
API_KEY = 'ur keySECRET_KEY = 'ur secretORDERS_URL = '{}/v2/orders'.format(BASE_URL)
HEADERS = {'APCA-API-KEY-ID':API_KEY,'APCA-API-SECRET-KEY':SECRET_KEY}

model_dir = 'C:\\Users\\85762\\OneDrive\\Desktop\\FINM 325'
model_file = model_dir + '\\Stocks_best_model.h5'

logging.basicConfig(filename='trading_log.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

while True:
    try:
        logging.info("Starting trading cycle.")

        # Step 1: Data Retrieval
        X, true_y, n_features, minmax, n_steps, close, open_, high, low = get_data(symbol, data_len, seq_len)
        logging.info("Data retrieved successfully.")

        # Step 2: Check if Model Exists, Load or Train Model
        if os.path.exists(model_file):
            # Load the pre-trained model
            model = load_model(symbol, Sequential, 'mse', 'Adam')
            logging.info("Loaded pre-trained model.")
        else:
            # Train a new model
            model = run_lstm(n_steps, n_features)
            X_train, X_test, y_train, y_test = split_data(X, true_y)
            test_loss = model.evaluate(X_test, y_test, verbose=0)
            history = train_model(X_train, y_train, model, epochs)
            logging.info("New model trained.")

        # Step 3: Make Prediction
        prediction, predict_loss = model_predict(model, minmax, n_features, n_steps, X, test_loss)
        logging.info(f"Prediction made: {prediction}")

        # Step 4: Execute Trade
        create_order(prediction, symbol, test_loss, predict_loss)
        logging.info("Trade executed.")

        # Step 5: Wait
        time.sleep(interval)

    except Exception as e:
        logging.error(f"An error occurred: {e}")